In [1]:
!pip3 install torch==1.7.1
!pip3 install tfrecord

  Using cached https://files.pythonhosted.org/packages/90/4f/acf48b3a18a8f9223c6616647f0a011a5713a985336088d7c76f3a211374/torch-1.7.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/ca/75fac5856ab5cfa51bbbcefa250182e50441074fdc3f803f6e76451fab43/dataclasses-0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/7a/e881b5abb54db0e6e671ab088d079c57ce54e8a01a3ca443f561ccadb37e/typing_extensions-3.7.4.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/fd/247ef25f5ec5f9acecfbc98ca3c6aaf66716cf52509aca9a93583d410493/protobuf-3.14.0-cp36-cp36m-manylinux1_x86_64.whl
  Usin

In [219]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from tfrecord.torch.dataset import TFRecordDataset
import tensorflow as tf
import math
import numpy as np

In [453]:
class Encoder(nn.Module):
    def __init__(self,d_model, nhead, dff, nlayers, size_embedding, dropout = 0):
        super(Encoder, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.dense1 = nn.Linear(3, d_model)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, dff, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.dense2 = nn.Linear(d_model, size_embedding)
        #self.init_weights()
    
    
    def get_last_time_step(self, tensor, stroke_lengths):
        
        embeddingd_lt = []
        
        for pos_embedding in range(tensor.shape[0]):
            embedding = tensor[pos_embedding, stroke_lengths[pos_embedding]-1,:]
            embeddingd_lt.append(embedding)
        
        embeddingd_lt = torch.vstack(embeddingd_lt) 
        
        return embeddingd_lt
    
    
    def forward(self, src, stroke_lengths, src_mask):
        #src = self.pos_encoder(src)
        output = self.dense1(src)
        output = self.transformer_encoder(output, src_mask)
     
        output = self.get_last_time_step(output, stroke_lengths)
        
        output = self.dense2(output)
        
        return output

In [454]:
path_data = "/data/jcabrera/didi_wo_text/training/"
name_file = "diagrams_wo_text_20200131-00000-of-00010"

In [455]:
tfrecord_path = path_data + name_file
index_path = None

In [456]:
description = {"key": "byte", "label_id":"byte",
               "ink":"float", "stroke_length":"int", "shape":"int", "num_strokes":"int", 
               "rdp_ink":"float", "rdp_stroke_length":"int", 
               "rdp_shape":"int", "rdp_num_strokes":"int",
               }

In [457]:
dataset = TFRecordDataset(tfrecord_path, index_path, description)
loader = torch.utils.data.DataLoader(dataset, batch_size=1)

In [458]:
dataiter = iter(loader)
data = dataiter.next()
print(data)

{'key': tensor([[ 48,  48,  51,  48,  55,  56,  57,  97,  57,  54,  97,  97, 101,  55,
         101,  97]], dtype=torch.uint8), 'label_id': tensor([[101,  50,  51,  56,  56,  97, 100,  56,  55,  98, 102,  55,  97,  52,
          98,  54,  49,  97,  51,  99,  52,  97,  55,  52, 102,  51,  49,  53,
          48, 101,  51, 100,  98,  97,  57,  56,  49, 100,  49,  97]],
       dtype=torch.uint8), 'ink': tensor([[0.1517, 0.5780, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]), 'stroke_length': tensor([[ 32,  32,  25,  43, 128,  31,  33,  34,  29]], dtype=torch.int32), 'shape': tensor([[  9, 128,   4]], dtype=torch.int32), 'num_strokes': tensor([[9]], dtype=torch.int32), 'rdp_ink': tensor([[187.4000, 751.9100,   0.0000,  ...,   0.0000,   0.0000,   0.0000]]), 'rdp_stroke_length': tensor([[38, 36, 33, 21, 29, 19, 16, 13, 16]], dtype=torch.int32), 'rdp_shape': tensor([[ 9, 38,  4]], dtype=torch.int32), 'rdp_num_strokes': tensor([[9]], dtype=torch.int32)}


In [459]:
#d model cantidad de features: 64
#
xd = data["ink"].reshape((9,128,4))[:,:,0:3]
stroke_lengths = data["stroke_length"].numpy().reshape(-1)

In [460]:
stroke_lengths.reshape(-1)[0]

32

In [461]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [462]:
encoder = Encoder(d_model=64, nhead=4, dff=128, nlayers=6, size_embedding=256, dropout = 0)

In [463]:
encoder = encoder.to(device)
xd = xd.to(device)

In [464]:
output = encoder(xd, None, stroke_lengths)

In [466]:
xd.shape

torch.Size([9, 128, 3])

In [467]:
xd

tensor([[[1.5168e-01, 5.7804e-01, 0.0000e+00],
         [1.3512e-01, 5.8202e-01, 2.5000e+01],
         [1.2253e-01, 5.8489e-01, 4.5000e+01],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[1.1419e+00, 8.1085e-02, 9.9100e+02],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[2.0582e+00, 6.3554e-01, 1.9720e+03],
         [2.0203e+00, 6.3852e-01, 1.9940e+03],
         [1.9913e+00, 6.4745e-01, 2.0140e+03],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        ...,

        [[1.5922e+00, 7.5728e-01, 8.1820e+03],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]

In [468]:
output

tensor([[-0.6174, -0.0596, -0.6434,  ...,  0.5592, -0.4438,  1.2578],
        [-0.6174, -0.0596, -0.6434,  ...,  0.5592, -0.4438,  1.2578],
        [-0.7713,  0.1280, -0.3362,  ...,  0.3897, -0.3782,  1.0778],
        ...,
        [-0.6170, -0.0603, -0.6431,  ...,  0.5592, -0.4434,  1.2571],
        [-0.6166, -0.0610, -0.6429,  ...,  0.5593, -0.4430,  1.2565],
        [-0.7245,  0.0429, -0.4661,  ...,  0.4466, -0.3874,  1.1754]],
       device='cuda:1', grad_fn=<AddmmBackward>)

In [469]:
output.shape

torch.Size([9, 256])

In [139]:
#xd = xd.type(torch.LongTensor)

In [140]:
xd

tensor([[[0.1517, 0.5780],
         [0.1351, 0.5820],
         [0.1225, 0.5849],
         ...,
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000]],

        [[1.1419, 0.0811],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         ...,
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000]],

        [[2.0582, 0.6355],
         [2.0203, 0.6385],
         [1.9913, 0.6474],
         ...,
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000]],

        ...,

        [[1.5922, 0.7573],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         ...,
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000]],

        [[0.6347, 0.8220],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         ...,
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000]],

        [[0.6258, 0.7169],
         [0.6520, 0.7122],
         [0.6952, 0.7096],
         ...,
 

In [125]:
embedding = nn.Embedding(50,64)

In [129]:
embedding_xd = embedding(xd)

In [100]:
!pip install --upgrade pip

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: pip in /home/shuaman/.local/lib/python2.7/site-packages (20.3.3)


In [116]:
device = "cuda:1"

In [113]:
encoder_layer = nn.TransformerEncoderLayer(d_model=64, nhead=4, dim_feedforward=128 ,dropout=0)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)

In [114]:
transformer_encoder.to(device)

TransformerEncoder(
  (layers): ModuleList(
    (0): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
      )
      (linear1): Linear(in_features=64, out_features=128, bias=True)
      (dropout): Dropout(p=0, inplace=False)
      (linear2): Linear(in_features=128, out_features=64, bias=True)
      (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0, inplace=False)
      (dropout2): Dropout(p=0, inplace=False)
    )
    (1): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
      )
      (linear1): Linear(in_features=64, out_features=128, bias=True)
      (dropout): Dropout(p=0, inplace=False)
      (linear2): Linear(in_features=128, out_features=64, bias=True)
      (norm1): LayerNorm((64,), ep

In [142]:
densa = nn.Linear(2, 64)
output = densa(xd)

In [143]:
output.shape

torch.Size([9, 128, 64])

In [145]:
output = output.to(device)

In [146]:
embedding_xd = embedding_xd.to(device)

In [147]:
output = transformer_encoder(embedding_xd, None)

In [149]:
output.shape

torch.Size([9, 128, 64])

In [151]:
dens2 = nn.Linear(64, 100)
dens2 = dens2.to(device)
output2 = dens2(output)

In [153]:
output2.shape

torch.Size([9, 128, 100])